In [1]:
from __future__ import absolute_import, division, unicode_literals
import sys
import io
import numpy as np
import logging
import argparse
import torch
import random
from transformers import *
import utils
import json
import pandas as pd
from tqdm import tqdm


/root/anaconda3/envs/SBERT-WK/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sbert import sbert

loading configuration file https://huggingface.co/binwang/bert-base-nli-stsb/resolve/main/config.json from cache at ./cache/741df96bc03932e0eade8d5031035bca0d2958179d96a37e973a5e1863450ccb.c5a70c2b03d4ec41c3610c8a99080e27c87d4599b28c5857597c193fc608bdfe
Model config BertConfig {
  "_name_or_path": "binwang/bert-base-nli-stsb",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file https://huggingface.co/binwan

to device


In [5]:
torch.device("cuda", 1)
classifier = pipeline("text-classification", model = "microsoft/deberta-xlarge-mnli")

loading configuration file https://huggingface.co/microsoft/deberta-xlarge-mnli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1c2adfd60f19f0beaf52e3c7988db42be627c0e366303902fec8997eed74b95f.4b3a81a93b125c8a71c26ca387b3068c017005c54e607aa7dd0b49d47862e204
Model config DebertaConfig {
  "_name_or_path": "microsoft/deberta-xlarge-mnli",
  "architectures": [
    "DebertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "CONTRADICTION",
    "1": "NEUTRAL",
    "2": "ENTAILMENT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "CONTRADICTION": 0,
    "ENTAILMENT": 2,
    "NEUTRAL": 1
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 48,
  "pad_token_id": 0,
  "pooler_dropout": 0,
 

In [3]:
import itertools
# Get max sim between 1 sentence and others
def get_max_sim(sentence, sentences):
    max_sim = 0
    for pair in itertools.product(sentence,sentences):
        sim = sbert([pair[0],pair[1]])
        if sim > max_sim:
            max_sim = sim
    return max_sim

In [4]:
import itertools
# Get max sim between 1 sentence and others
def get_mean_sim(sentence, sentences):
    mean_sim = []
    for pair in itertools.product(sentence,sentences):
        sim = sbert([pair[0],pair[1]])
        mean_sim.append(sim)
    return np.mean(mean_sim)

In [20]:
# df = pd.read_json('/root/thesis/ViLT/cosmos/test_data.json',orient="records", lines=True)

In [6]:
df = pd.read_csv('new_data.csv')

In [7]:
nli_data = pd.read_json('/root/thesis/SemBERT/nli-data.json',orient="records")

In [7]:
tqdm.pandas()
df['sbert_wk_score_m'] = df.progress_apply(lambda x: sbert([x.caption1_modified,x.caption2_modified]), axis=1)

100%|██████████| 1700/1700 [00:38<00:00, 44.28it/s]


In [8]:
tqdm.pandas()
df['nli_label'] = nli_data['nli_label']
df['nli_label_reverse'] = nli_data['nli_label_reverse']

In [39]:
converter = lambda x : 'CONTRADICTION' if (x.nli_label['label'] == 'CONTRADICTION' and x.nli_label_reverse['label'] == 'CONTRADICTION')else ('ENTAILMENT' if (x.nli_label['label'] == 'ENTAILMENT' and x.nli_label_reverse['label'] == 'ENTAILMENT') else 'NEUTRAL')
df['nli'] = df.progress_apply(converter, axis=1) 

100%|██████████| 1700/1700 [00:00<00:00, 29470.82it/s]


In [9]:
converter = lambda x : 'CONTRADICTION' if (x.nli_label['label'] == 'CONTRADICTION' and x.nli_label_reverse['label'] == 'CONTRADICTION') else ('ENTAILMENT' if (x.nli_label['label'] == 'ENTAILMENT' or x.nli_label_reverse['label'] == 'ENTAILMENT') else 'NEUTRAL')
df['nli'] = df.progress_apply(converter, axis=1) 

100%|██████████| 1700/1700 [00:00<00:00, 22481.69it/s]


In [56]:
converter = lambda x : 'CONTRADICTION' if (x.nli_label['label'] == 'CONTRADICTION' or x.nli_label_reverse['label'] == 'CONTRADICTION') else ('ENTAILMENT' if (x.nli_label['label'] == 'ENTAILMENT' or x.nli_label_reverse['label'] == 'ENTAILMENT') else 'NEUTRAL')
df['nli'] = df.progress_apply(converter, axis=1) 

100%|██████████| 1700/1700 [00:00<00:00, 21404.26it/s]


In [12]:
df['sbert_wk_score'] = df['bert_base_score']

In [10]:
df['caption_3'] = df['caption_3'].apply( lambda x: x.strip("[]").replace("'","").split(", "))

In [15]:
tqdm.pandas()
df['sbert_wk_score_1_3'] = df.progress_apply(lambda x: 
get_mean_sim([x.caption1_modified],x.caption_3)
, axis=1)
df['sbert_wk_score_2_3'] = df.progress_apply(lambda x: 
get_mean_sim([x.caption2_modified],x.caption_3)
, axis=1)

100%|██████████| 1700/1700 [01:55<00:00, 14.68it/s]


In [22]:
def get_false_scores(row):
    new_scores = [0,0,0,0]
    c1_fake = row['caption1'][:-1] + ' is not genuine.'
    new_scores[0] = sbert([c1_fake,row['caption2']])

    c1_fake = row['caption1'][:-1] + ' is fake.'
    new_scores[1] = sbert([c1_fake,row['caption2']])

    c1_fake = row['caption1'][:-1] + ' wasn\'t true.'
    new_scores[2] = sbert([c1_fake,row['caption2']])

    c1_fake = 'No, ' + row['caption1'][:-1]
    new_scores[3] = sbert([c1_fake,row['caption2']])
    return new_scores
df['false_scores'] = df.progress_apply(lambda x: get_false_scores(x), axis=1)

100%|██████████| 1700/1700 [02:39<00:00, 10.69it/s]


In [ ]:
def is_fake(row):
    sen = "fake, hoax, fabrication, supposedly, falsification, propaganda, deflection, deception, contradicted, defamation, lie, misleading, deceive, fraud, concocted, bluffing, made up, double meaning, alternative facts, trick, half-truth, untruth, falsehoods, inaccurate, disinformation, misconception"
    new_scores = [0,0]
    new_scores[0] = sbert([row['caption1'],sen])
    new_scores[1] = sbert([row['caption2'],sen])
    if new_scores[0] < 0:
       new_scores[0] = 0.0001
    if new_scores[1] < 0:
       new_scores[1] = 0.0001
    return (new_scores[0] < 0.1 and new_scores[1] > 0.15) or (new_scores[1] < 0.1 and new_scores[0] > 0.15)
df['fake_predict'] = df.progress_apply(lambda x: is_fake(x), axis=1)

100%|██████████| 1700/1700 [01:21<00:00, 20.92it/s]


In [14]:
df['bb']= pd.read_csv('/root/thesis/COSMOS/pred_contexts.txt', header=None)[0]

In [38]:
def predict(row):
    if row.nli == 'CONTRADICTION':
        return True
    false_scores = row['false_scores']
    if false_scores[0] > row['sbert_wk_score']+0.05:
        return True
    if false_scores[1] > row['sbert_wk_score']+0.05:
        return True
    if false_scores[2] > row['sbert_wk_score']+0.05:
        return True
    if false_scores[3] > row['sbert_wk_score']+0.05:
        return True
    # if row['fake_predict']==True:
    #     return True
    if row.nli == 'ENTAILMENT':
        return False

    if row['sbert_wk_score']>0.5:
        # if row['sbert_wk_score']>0.7:
        #     if false_scores[0] > row['sbert_wk_score']+0.02:
        #         return True
        #     if false_scores[1] > row['sbert_wk_score']+0.02:
        #         return True
        #     if false_scores[2] > row['sbert_wk_score']+0.02:
        #         return True
        #     if false_scores[3] > row['sbert_wk_score']+0.02:
        #         return True
        # 52 cau similar, khong doi lap nhung OOC
        # if row['context_label']==1:
        #     print('g')
        return False
    
    # # if row['sbert_wk_score_1_3'] > 0.5 or row['sbert_wk_score_2_3'] > 0.5 :
    # #     # 18 cau khong doi lap, captiong match nhung OOC
    # #     return False
    # # 26 cau sai nhung no thay dung
    # # 75 cau dung nhung no thay sai
    # # Tong cau o day la 569. 17% cau o day sai
    return row['bb']==True
    # # if row['iou']<0.5:
    # #     return False

    # return True

In [39]:
# Language Baseline
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual       0    1
Predicted          
False      701   67
True       149  783
accuracy: 0.8729411764705882


In [67]:
predict = df[df['predict'] == 1]
predict = predict[predict['context_label'] == 0]
predict

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,...,iou,nli_label,nli_label_reverse,nli,sbert_wk_score,sbert_wk_score_1_3,sbert_wk_score_2_3,false_scores,bb,predict
9,test/9.jpg,A man brushes the mouth of a sarcophagus.,More than 40 statues of ancient deities and fu...,0,https://www.bbc.com/news/world-africa-55002070,"[[131.27796936035156, 0.8781880140304561, 736....",A man brushes the mouth of a sarcophagus.,[],CARDINAL statues of ancient deities and funera...,"[['More than 40', 'CARDINAL']]",...,0.621256,"{'label': 'NEUTRAL', 'score': 0.9956753850000001}","{'label': 'NEUTRAL', 'score': 0.6785371304000001}",NEUTRAL,0.219293,0.271516,0.290554,"[0.239371, 0.2798361, 0.21128304, 0.13894248]",False,True
54,test/54.jpg,British Prime Minister Theresa May addresses t...,Prime minister Theresa May delivers a statemen...,0,https://www.theguardian.com/artanddesign/galle...,"[[476.4560241699219, 386.9877014160156, 552.15...",NORP Prime Minister PERSON addresses the media...,"[['British', 'NORP'], ['Theresa May', 'PERSON'...",Prime minister PERSON delivers a statement to ...,"[['Theresa May', 'PERSON'], ['Downing Street',...",...,0.733261,"{'label': 'NEUTRAL', 'score': 0.6677771211}","{'label': 'NEUTRAL', 'score': 0.6465447545}",NEUTRAL,0.397588,0.178863,0.016378,"[0.40628433, 0.36568594, 0.43237096, 0.46672466]",False,True
64,test/64.jpg,Indian paramilitary troops in Srinagar in Kash...,"Kashmir braces for worst as shelling, gunbattl...",0,https://www.nytimes.com/2019/03/03/world/asia/...,"[[154.84408569335938, 262.2037658691406, 194.3...",NORP paramilitary troops in GPE in GPE DATE. P...,"[['Indian', 'NORP'], ['Srinagar', 'GPE'], ['Ka...","Kashmir braces for worst as shelling, gunbattl...",[],...,0.467560,"{'label': 'NEUTRAL', 'score': 0.9469358325}","{'label': 'NEUTRAL', 'score': 0.8719753623000001}",NEUTRAL,0.238985,0.028563,-0.038022,"[0.31643695, 0.2924672, 0.26093918, 0.23317963]",True,True
78,test/78.jpg,A controlled detonation was carried out after ...,photo shows the unexploded bomb with their hou...,0,https://www.bbc.com/news/in-pictures-56283009,"[[403.8778076171875, 612.6676025390625, 510.85...",A controlled detonation was carried out after ...,"[['World War Two', 'EVENT'], ['England', 'GPE']]",photo shows the unexploded bomb with their hou...,[],...,0.693168,"{'label': 'NEUTRAL', 'score': 0.9301253557}","{'label': 'NEUTRAL', 'score': 0.9664840698}",NEUTRAL,0.515037,0.211392,0.171085,"[0.5117989, 0.56768245, 0.47552994, 0.5438447]",True,True
97,test/97.jpg,An old man leans up against his truck in Granby.,East Troublesome fire grows as residents evacu...,0,https://www.denverpost.com/2020/10/22/photos-e...,"[[325.7146301269531, 173.37582397460938, 395.4...",An old man leans up against his truck in GPE.,"[['Granby', 'GPE']]",East Troublesome fire grows as residents evacu...,"[['Granby', 'GPE']]",...,0.357074,"{'label': 'CONTRADICTION', 'score': 0.77687680...","{'label': 'CONTRADICTION', 'score': 0.65027189...",CONTRADICTION,0.147661,0.518331,0.021240,"[0.24066299, 0.17455386, 0.22583605, 0.12034923]",False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1611,test/1611.jpg,India clears purchase of Russian combat jets w...,"Kashmir braces for worst as shelling, gunbattl...",0,https://www.nytimes.com/2019/03/03/world/asia/...,"[[154.84408569335938, 262.2037658691406, 194.3...",GPE clears purchase of NORP combat jets worth ...,"[['India', 'GPE'], ['Russian', 'NORP'], ['$2.4...","Kashmir braces for worst as shelling, gunbattl...",[],...,0.441132,"{'label': 'NEUTRAL', 'score': 0.5764892697}","{'label': 'NEUTRAL', 'score': 0.8466338515}",NEUTRAL,0.017282,-0.080160,-0.010270,"[0.1136597, 0.072233535, 0.054292392, 0.020836...",False,True
1638,test/1638.jpg,The F.B.I. investigated whether the firing of ...,James Comey was fired in May 2017 - just month...,0,https://www.nytimes.com/2019/01/11/us/

In [14]:
# Language Baseline + neu phu dinh
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual       0    1
Predicted          
False      571   52
True       279  798
accuracy: 0.8052941176470588


In [29]:
save = df[df['predict'] != df['context_label']]
save.to_json('new_false_cases.json', orient='records', lines=True)

In [33]:
save

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,...,bbs2,iou,nli_label,nli_label_reverse,nli,sbert_wk_score,bb,predict,fake_predict,false_scores
9,test/9.jpg,A man brushes the mouth of a sarcophagus.,More than 40 statues of ancient deities and fu...,0,https://www.bbc.com/news/world-africa-55002070,"[[131.27796936035156, 0.8781880140304561, 736....",A man brushes the mouth of a sarcophagus.,[],CARDINAL statues of ancient deities and funera...,"[['More than 40', 'CARDINAL']]",...,"[[0.4474790692329407, 0.5808447599411011, 0.46...",0.621256,"{'label': 'NEUTRAL', 'score': 0.9956753850000001}","{'label': 'NEUTRAL', 'score': 0.6785371304000001}",NEUTRAL,0.219293,False,True,True,"[0.239371, 0.2798361, 0.21128304, 0.13894248]"
16,test/16.jpg,"On the same day in South Africa, a bird walks ...",General view of an empty beach with high-rise ...,0,https://www.bbc.com/news/world-africa-55573602,"[[275.4281311035156, 345.242431640625, 296.386...","On DATE in GPE, a bird walks along Durban's em...","[['the same day', 'DATE'], ['South Africa', 'G...",General view of an empty beach with high-rise ...,[],...,"[[0.49963846802711487, 0.5366191267967224, 0.9...",0.530790,"{'label': 'NEUTRAL', 'score': 0.8851500750000001}","{'label': 'NEUTRAL', 'score': 0.9272819757}",NEUTRAL,0.447258,True,True,False,"[0.32722235, 0.41247627, 0.30772707, 0.47252345]"
28,test/28.jpg,On Tuesday police patrol the school where more...,Police patrol inside the Government Science sc...,0,https://www.bbc.com/news/world-africa-55346788,"[[640.4035034179688, 120.90727233886719, 694.2...",On DATE police patrol the school where CARDINA...,"[['Tuesday', 'DATE'], ['more than 300', 'CARDI...","Police patrol inside the ORG school in GPE, in...","[['Government Science', 'ORG'], ['Kankara', 'G...",...,"[[0.5371631979942322, 0.33739516139030457, 0.4...",0.685403,"{'label': 'NEUTRAL', 'score': 0.9495272636000001}","{'label': 'NEUTRAL', 'score': 0.9873914123}",NEUTRAL,0.445600,True,True,False,"[0.39261526, 0.42360815, 0.40280885, 0.45446125]"
48,test/48.jpg,"The Cow and the Tailor Shop, Rajasthan, by Eli...",A sacred cow in a street in India,0,https://www.bbc.com/news/in-pictures-53234985,"[[704.8804931640625, 359.57080078125, 856.7168...","The Cow and the Tailor Shop, PERSON, by PERSON...","[['Rajasthan', 'PERSON'], ['Elizabeth Brooks',...",A sacred cow in a street in GPE,"[['India', 'GPE']]",...,"[[0.498828649520874, 0.5964764952659607, 0.473...",0.576261,"{'label': 'NEUTRAL', 'score': 0.6408179402}","{'label': 'NEUTRAL', 'score': 0.8794209361}",NEUTRAL,0.487040,True,True,False,"[0.48298848, 0.45156243, 0.4697553, 0.49902934]"
54,test/54.jpg,British Prime Minister Theresa May addresses t...,Prime minister Theresa May delivers a statemen...,0,https://www.theguardian.com/artanddesign/galle...,"[[476.4560241699219, 386.9877014160156, 552.15...",NORP Prime Minister PERSON addresses the media...,"[['British', 'NORP'], ['Theresa May', 'PERSON'...",Prime minister PERSON delivers a statement to ...,"[['Theresa May', 'PERSON'], ['Downing Street',...",...,"[[0.5457073450088501, 0.61725252866745, 0.3729...",0.733261,"{'label': 'NEUTRAL', 'score': 0.6677771211}","{'label': 'NEUTRAL', 'score': 0.6465447545}",NEUTRAL,0.397588,False,True,False,"[0.40628433, 0.36568594, 0.43237096, 0.46672466]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,test/1676.jpg,"How Julia Jordan, Champion of Female Playwrigh...","“I’m primarily the dog walker, but usually the...",0,https://www.nytimes.com/2018/10/26/nyregion/ho...,"[[165.6656494140625, 109.18206024169922, 267.9...","How Julia PERSON, Champion of Female Playwrigh...","[['Jordan', 'PERSON']]","“I’m primarily the dog walker, but usually the...",[],...,"[[0.4680388569831848, 0.8286891579627991, 0.43...",0.768620,"{'label': 'NEUTRAL', 'score': 0.795879662}","{'label': 'NEUTRAL', 'score': 0.7681475282}",NEUTRAL,0.1595

In [32]:
df[943:944]

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,...,bbs2,iou,nli_label,nli_label_reverse,nli,sbert_wk_score,bb,predict,fake_predict,false_scores
943,test/943.png,A photograph shows Donald Trump posing with hi...,This isn’t even the first time that Ku Klux Kl...,1,https://www.snopes.com/fact-check/donald-trump...,"[[276.6856994628906, 175.16421508789062, 321.8...",A photograph shows PERSON posing with his pare...,"[['Donald Trump', 'PERSON'], ['Ku Klux Klan', ...",This isn’t even the ORDINAL time that ORG robe...,"[['first', 'ORDINAL'], ['Ku Klux Klan', 'ORG']...",...,"[[0.48939329385757446, 0.5332639813423157, 0.8...",0.654888,"{'label': 'NEUTRAL', 'score': 0.9861732721}","{'label': 'NEUTRAL', 'score': 0.8068803549}",NEUTRAL,0.379469,False,False,False,"[0.39190748, 0.36802748, 0.40497515, 0.35911334]"


In [16]:
# Language Baseline + neu phu dinh + image_caption threshold 0.5
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual       0    1
Predicted          
False      621   75
True       229  775
accuracy: 0.8211764705882353


In [11]:
# Language Baseline + neu phu dinh + image_caption + cosmos
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)
#0.808

Actual       0    1
Predicted          
False      722  101
True       128  749
accuracy: 0.8652941176470588


In [31]:
# entailment + neu phu dinh
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)
#0.808

Actual       0    1
Predicted          
False      685   47
True       165  803
accuracy: 0.8752941176470588


In [39]:
## misleading words (is fake, is not genuine,...) => không bias
# entailment + neu phu dinh
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)
#0.808

Actual       0    1
Predicted          
False      707   64
True       143  786
accuracy: 0.8782352941176471


In [43]:
df['caption_3'].head(3)

0    [a man standing at a podium in front of a crow...
1    [man sitting on the top of an unmingy in front...
2    [a group of people standing around a table wit...
Name: caption_3, dtype: object